In [35]:
import numpy as np
import pandas as pd

In [37]:
import pandas as pd
import psycopg2
import pandas.io.sql as sqlio
import requests
import pickle
import config as cfg

class Data_Gathering:
    
    def __init__(self, df):
        self.df = df
        
    def get_table(self, sql_str):
        '''
        query the database and return a dataframe with given sql query string
        input: `sql_str`, query string
        return: a dataframe
        '''
        host= cfg.db['host']
        port = cfg.db['port']
        user = cfg.db['user']
        pwd = cfg.db['pwd']
        database= cfg.db['database']

        con = psycopg2.connect(database=database, user=user, password=pwd, host=host, port=port)

        print("Database opened successfully")

        sql = "'''" + sql_str + "'''"
        df = sqlio.read_sql_query(sql, con)

        return df
    
    def get_weather(self, df):
        '''
        request weather information
        input: a data frame in which
            - first column: date
            - second column: location in "latitude, longitude"        
        output: a dictionary 
        '''
        key = cfg.rapid_api['key']
        base_url = "https://dark-sky.p.rapidapi.com"
        headers = { 'x-rapidapi-host': "dark-sky.p.rapidapi.com",
                    'x-rapidapi-key': key }
        weather_dict = { 'weather':[] }

        count = 1
        for row in df.values:
            print('Query Count: ', count)
            date = row[0]
            ll = row[1]
            time = row[2]
            url = base_url + '/' + ll + ',' + date + 'T' + time        
            res = requests.get(url, headers=headers)
            if res.status_code == 200:
                weather_json = res.json()
                try:
                    weather = weather_json['currently']['summary']
                except:
                    weather = 'NA'
                finally:        
                    weather_dict['weather'].append(weather)
            else:
                weather_dict['weather'].append('NA')            
            count += 1
        return weather_dict


In [38]:
df = pd.read_csv('../../data/flights_samp.csv')
df_air = pd.read_csv('../../data/airports_usa.csv')

In [10]:
df_air = df_air[['IATA_CODE', 'LATITUDE', 'LONGITUDE']]
df_air
# df_air = df_air.rename(columns={'IATA_CODE': 'origin'})

,IATA_CODE,LATITUDE,LONGITUDE
0,ABE,40.65236,-75.44040
1,ABI,32.41132,-99.68190
2,ABQ,35.04022,-106.60919
3,ABR,45.44906,-98.42183
4,ABY,31.53552,-84.19447
...,...,...,...
317,WRG,56.48433,-132.36982
318,WYS,44.68840,-111.11764
319,XNA,36.28187,-94.30681
320,YAK,59.50336,-139.66023


In [11]:
top20_airport_code = ['LAX', 'ORD', 'EWR', 'SFO', 'LGA', 'DFW', 'LAS', 'CLT', 'DEN',
                      'PHL', 'IAH', 'SEA', 'ATL', 'PHX', 'MCO', 'DTW', 'SLC', 'BOS',
                      'JFK', 'MSP']

In [15]:
import help_functions as hf

In [95]:
df = df.merge(df_air, on='origin', how='left')
df['ll'] = df.LATITUDE.astype('str') + ',' + df.LONGITUDE.astype('str')
df.drop(columns=['LATITUDE', 'LONGITUDE'], inplace=True)

In [96]:
df_origin = df[df.origin.isin(top20_airport_code)]
df_origin = df_origin [['fl_date', 'll']]
df_origin = df_origin.drop_duplicates().reset_index(drop=True)

In [17]:
df.columns

Index(['index', 'fl_date', 'mkt_unique_carrier', 'branded_code_share',
       'mkt_carrier', 'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name'],
      dtype='object')

In [79]:
def get_avg_delay(df, col_list):
    df.loc[:, col_list] = df.loc[:, col_list].fillna(0)
    for col in col_list:
        s = df.groupby('dest')[col].mean()        
        s.name = 'avg_' + col        
        df = df.merge(s.to_frame(), on='dest', how='left')
    return df

In [80]:
delay_col_list= ['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay', 'first_dep_time']

In [81]:
get_avg_delay(df, col_list)

,index,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,...,first_dep_time,total_add_gtime,longest_add_gtime,no_name,avg_carrier_delay,avg_weather_delay,avg_nas_delay,avg_security_delay,avg_late_aircraft_delay,avg_first_dep_time
0,244649,2019-01-07,AA,AA,AA,362,AA,N161AA,362,12892,...,0.0,NaN,NaN,NaN,5.937238,0.158996,3.987448,0.037657,7.288703,18.062762
1,190528,2019-01-04,UA,UA_CODESHARE,UA,3788,ZW,N437AW,3788,13930,...,0.0,NaN,NaN,NaN,0.000000,16.000000,0.000000,0.000000,3.333333,383.666667
2,50595,2018-12-29,WN,WN,WN,5741,WN,N7738A,5741,13871,...,0.0,NaN,NaN,NaN,3.953271,0.654206,1.859813,0.457944,6.775701,0.000000
3,116214,2019-01-01,WN,WN,WN,1641,WN,N423WN,1641,15304,...,0.0,NaN,NaN,NaN,9.085714,0.000000,2.600000,0.000000,8.542857,0.000000
4,134474,2019-01-02,UA,UA_CODESHARE,UA,4233,EV,N14558,4233,11618,...,0.0,NaN,NaN,NaN,4.323077,0.123077,6.384615,0.000000,4.507692,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,118168,2019-01-01,DL,DL,DL,2173,DL,N535US,2173,12892,...,0.0,NaN,NaN,NaN,6.173913,0.119565,0.184783,0.000000,1.826087,12.043478
4996,306574,2019-01-10,G4,G4,G4,1608,G4,253NV,1608,14082,...,0.0,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4997,303601,2019-01-10,DL,DL_CODESHARE,DL,3289,9E,N294PQ,3289,11003,...,0.0,NaN,NaN,NaN,6.093878,0.187755,1.779592,0.016327,4.612245,0.000000
4998,346537,2019-01-12,AA,AA,AA,2809,AA,N338RS,2809,12953,...,0.0,NaN,NaN,NaN,3.431034,0.000000,1.982759,0.000000,2.017241,0.000000


In [82]:
df.columns

Index(['index', 'fl_date', 'mkt_unique_carrier', 'branded_code_share',
       'mkt_carrier', 'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name'],
      dtype='object')